In [142]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import censusdata
import re
import geopandas
import sqlite3
from sqlite3 import Error

censuskey = '7b3e8a6afe58867abcf5aecad26845d802115f1f'

# The following section will import Census dataset

In [143]:
#Define Census Funtions

#Funcion For NonDigit Removing Characters from Row - Used for Extracting GEOID
def remove_chars(s):
    clean = int(re.sub('[^0-9]+', '', str(s))[-11:])
    return clean

#Function to get census data according to requirements
def getcensustables(table):
    tablename = 'data_' + table
    print(f'Downloading {table}')
    
    # Set requirements to pull from Census API
    
    variablestopull = allvariablescsv.loc[allvariablescsv['Group']==table]['Name'].tolist()
    listofcounties = ['121', '089']
    yearstopull = [2017] #range(2018,2018,1)

    #Pull Census Data according to requirement above
    atlantadata = pd.DataFrame()

    for year in yearstopull:
        for county in listofcounties:

            newdata = censusdata.download('acs5', year, censusdata.censusgeo([('state', '13'), ('county', county), ('tract', '*')]),variablestopull,censuskey)
            type(newdata)
            newdata['YEAR'] = year
            atlantadata = atlantadata.append(newdata)

    #Rename Column Headings from Code to Text
    for variable in variablestopull:
        #print(variable)
        renamevar1 = allvariablescsv.loc[allvariablescsv['Name']==variable]['Label'].iloc[0]
        renamevar2 = allvariablescsv.loc[allvariablescsv['Name']==variable]['Concept'].iloc[0]
        renamevar = renamevar2 + '_'+ renamevar1
        #print(renamevar)
        atlantadata = atlantadata.rename(columns={variable: renamevar})

    #Transform Index to create GeoID

    #Create GeoID Column
    atlantadata['Name2'] = atlantadata.index
    atlantadata['GEOID'] = atlantadata['Name2'].apply(remove_chars)
    atlantadata.drop(columns=['Name2'], inplace=True)
    
    #Only Keep Atlanta Census Tracts(Geocodes)
    finaldata = atlantageocodes.merge(atlantadata, how='left', on='GEOID')
    
    finaldata = finaldata.dropna(axis=0, how='any')
    
    return finaldata

In [144]:
#Load tables already downloaded, or download if not.

#Tables to Load from data/tablestopull.csv
tablestoloadcsv = pd.read_csv('data/tablestopull.csv')
tablestoload = tablestoloadcsv['Tables'].tolist()

#Allvariables Table to get table detail
allvariablescsv = pd.read_csv('data/allvariables.csv')

#Atlanta Geo Codes Table to filter to city of atlanta only 
atlantageocodes = pd.read_csv('data/MetroATLCensusTracts.csv')

#Create dictionary with each cleaned table as an entry
alltables = dict()  


#Script will attempt to load tables from data folder first, otherwise download them
for table in tablestoload:
    try:
        data = pd.read_csv('data/' + table + '.csv')
        alltables[table] = data
        print(f'Loading {table}')
    except:
        data = getcensustables(table)
        alltables[table] = data

        data.to_csv('data/' + table + '.csv')
        
        

Loading B19013
Loading B01003
Loading B11001
Loading B25064
Loading B25074
Loading B25063
Loading B25061
Loading B25056
Loading B00002
Loading B25001
Loading B25122
Loading B19119
Loading B19019
Loading B25106


In [145]:
#Pull Housing Burdened Households
housingburdened = alltables['B25074']

housingburdenedhhs = pd.DataFrame()

# only keep the related columns
cols_of_sum=['HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!Less than 20.0 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!20.0 to 24.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!25.0 to 29.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!30.0 to 34.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!35.0 to 39.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!40.0 to 49.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!50.0 percent or more',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$10,000 to $19,999!!Less than 20.0 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$10,000 to $19,999!!20.0 to 24.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$10,000 to $19,999!!25.0 to 29.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$10,000 to $19,999!!30.0 to 34.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$10,000 to $19,999!!35.0 to 39.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$10,000 to $19,999!!40.0 to 49.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$10,000 to $19,999!!50.0 percent or more',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$20,000 to $34,999!!Less than 20.0 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$20,000 to $34,999!!20.0 to 24.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$20,000 to $34,999!!25.0 to 29.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$20,000 to $34,999!!30.0 to 34.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$20,000 to $34,999!!35.0 to 39.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$20,000 to $34,999!!40.0 to 49.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$20,000 to $34,999!!50.0 percent or more',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$35,000 to $49,999!!Less than 20.0 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$35,000 to $49,999!!20.0 to 24.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$35,000 to $49,999!!25.0 to 29.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$35,000 to $49,999!!30.0 to 34.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$35,000 to $49,999!!35.0 to 39.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$35,000 to $49,999!!40.0 to 49.9 percent',\
       'HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$35,000 to $49,999!!50.0 percent or more']

# lowincomehhs = percentrentbyincome[cols_of_interest]
housingburdenedhhs['GEOID'] = housingburdened['GEOID']
housingburdenedhhs['RentBurdenedHHs'] = housingburdened[cols_of_sum].sum(axis=1)

# clean_HUDcsv.rename(columns={'Code': 'GEOID'},inplace=True)
display(housingburdenedhhs.head())

,GEOID,RentBurdenedHHs
0,13089020100,82.0
1,13089020200,176.0
2,13089020300,225.0
3,13089020400,330.0
4,13089020500,642.0


In [146]:
#Pull Median Income per Household

medianincome = alltables['B19019']

medianincome.drop(columns='YEAR')

# display(medianincome)
#Set years to loop through
years = [2017.0]

medianincomehh = pd.DataFrame()

# lowincomehouseholds['TotalHHs'] = medianincomehh

# only keep the related columns
cols_of_sum=['MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS) BY HOUSEHOLD SIZE_Estimate!!Total']

# lowincomehhs = medianincomehh[cols_of_interest]
medianincomehh['GEOID'] = medianincome['GEOID']
medianincomehh['MedianIncomeHH'] = medianincome[cols_of_sum]

medianincomehh = medianincomehh.loc[medianincomehh['MedianIncomeHH'] > 0]

display(medianincomehh.head())

,GEOID,MedianIncomeHH
0,13089020100,174526.0
1,13089020200,87889.0
2,13089020300,113088.0
3,13089020400,78681.0
4,13089020500,55208.0


In [147]:
#Pull Percent Rent by Income
percentrentbyincome = alltables['B25106']

percentrentbyincome = percentrentbyincome[percentrentbyincome.columns.drop(list(percentrentbyincome.filter(regex='Owner')))]
percentrentbyincome = percentrentbyincome[percentrentbyincome.columns.drop(list(percentrentbyincome.filter(regex='percent')))]

# display(percentrentbyincome) ### for class

lowincomehhs = pd.DataFrame()

# only keep the related columns
cols_of_sum=['TENURE BY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Renter-occupied housing units!!Less than $20,000',\
                  'TENURE BY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Renter-occupied housing units!!$20,000 to $34,999',\
                  'TENURE BY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Renter-occupied housing units!!$35,000 to $49,999']

lowincomehhs['GEOID'] = percentrentbyincome['GEOID']
lowincomehhs['TotalLowIncomeHHs'] = percentrentbyincome[cols_of_sum].sum(axis=1)

display(lowincomehhs.head())

,GEOID,TotalLowIncomeHHs
0,13089020100,47.0
1,13089020200,132.0
2,13089020300,169.0
3,13089020400,304.0
4,13089020500,432.0


In [148]:
#Final Census Summary Table for DB

summary = housingburdenedhhs.merge(lowincomehhs.merge(medianincomehh, on='GEOID', how='inner'), on='GEOID', how='inner')

display(summary.head())
# actualsdata = actualsdata.merge(hweatherdata, on='date', how='inner')

,GEOID,RentBurdenedHHs,TotalLowIncomeHHs,MedianIncomeHH
0,13089020100,82.0,47.0,174526.0
1,13089020200,176.0,132.0,87889.0
2,13089020300,225.0,169.0,113088.0
3,13089020400,330.0,304.0,78681.0
4,13089020500,642.0,432.0,55208.0


# # The following section will import Georgia Census Tract dataset

In [149]:
#Read geotable
geotable = geopandas.read_file('data/tl_2018_13_tract.shp')

#Convert Geopandas table to DataFrame for import 
geotable = pd.DataFrame(geotable.drop(columns='geometry'))

#Convert GEOID to Int64
geotable.GEOID = geotable.GEOID.astype(str).astype('int64')
                                                   
#Keep only Atlanta GEOCODEs                                                   
geotable = atlantageocodes.merge(geotable, how='left', on='GEOID')

# Only keep the related columns
cols_of_interest=['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAMELSAD', 'INTPTLAT', 'INTPTLON']
geotableclean = geotable[cols_of_interest]
geotableclean.rename(columns={'STATEFP': 'StateID','COUNTYFP': 'CountyID','TRACTCE': 'TractID','NAMELSAD': 'TractName','INTPTLAT': 'Latitude','INTPTLON': 'Longitude'},inplace=True)
display(geotableclean.head())


,StateID,CountyID,TractID,GEOID,TractName,Latitude,Longitude
0,13,013,180103,13013180103,Census Tract 1801.03,+34.0791395,-083.8083173
1,13,013,180104,13013180104,Census Tract 1801.04,+34.0058079,-083.8339347
2,13,013,180105,13013180105,Census Tract 1801.05,+34.0405382,-083.8155021
3,13,013,180106,13013180106,Census Tract 1801.06,+34.0237319,-083.8424034
4,13,013,180107,13013180107,Census Tract 1801.07,+34.0149923,-083.7895842


# The following section will import Georgia HUD dataset

In [150]:
#Import the dataset
HUDcsv = pd.read_csv('data/hudPicture2018_363148.csv')


# Only keep the related columns
cols_of_interest=['Code','Subsidized units available','% Occupied','% Reported','Household income per year','% very low income','% extremely low income'
       ,'% female head','% with disability among Head Spouse Co-head aged 61 years or less','% with disability among Head Spouse Co-head aged 62 years or older',
       '% in poverty (Census tract)','% minority (Census tract)']
clean_HUDcsv = HUDcsv[cols_of_interest]
clean_HUDcsv.rename(columns={'Code': 'GEOID'},inplace=True)

#Convert GEOID to Int64

clean_HUDcsv = clean_HUDcsv[~clean_HUDcsv.GEOID.str.contains("XXX")]
clean_HUDcsv.GEOID = clean_HUDcsv.GEOID.astype(str).astype('int64')

#Keep only atlanta GEOCODES
clean_HUDcsv = atlantageocodes.merge(clean_HUDcsv, how='left', on='GEOID')

display(clean_HUDcsv.head())

,GEOID,Subsidized units available,% Occupied,% Reported,Household income per year,% very low income,% extremely low income,% female head,% with disability among Head Spouse Co-head aged 61 years or less,% with disability among Head Spouse Co-head aged 62 years or older,% in poverty (Census tract),% minority (Census tract)
0,13013180103,6,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4
1,13013180104,5,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4
2,13013180105,1,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4
3,13013180106,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,13013180107,4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4


# The following section will import the NHPD dataset

In [153]:
#Import dataset from CSV
income_data = "data\GA_Prop_NHPD.csv"
income_data_df = pd.read_csv(income_data)

subset_income_pd = income_data_df[['NHPD Property ID', 'PropertyStatus', 'Property Name', 'Property Address', 'City',
                                   'State', 'Zip', 'Census Tract', 'Active Subsidies',  
                                   'TargetTenantType','Occupancy Rate','0-1 BedroomUnits', 'TwoBedroomUnits',
                                   'ThreePlusBedroomUnits','Percent of ELI Households', 'HUD Property ID', 
                                   'FairMarketRent_2BR', 'Owner Type', 'EarliestStartDate', 'Total Units', 
                                    'S8_1_AssistedUnits', 'S202_1_AssistedUnits', 'FHA_1_AssistedUnits', 'FHA_2_AssistedUnits',
                                   'LIHTC_1_AssistedUnits', 'LIHTC_2_AssistedUnits', 'HOME_1_AssistedUnits', 'RHS515_1_AssistedUnits']]

subset_income_pd.rename(columns={'S202_1_AssistedUnits': 'Supportive Housing Elderly', 'RHS515_1_AssistedUnits': 'Rural Rental Housing', 'LIHTC_1_AssistedUnits' :
                                'Low Income Housing Tax Credit', 'LIHTC_2_AssistedUnits':'Low Income Housing Tax Credit sub-program',
                                 'S8_1_AssistedUnits':'Section 8 Housing Units', 'FHA_1_AssistedUnits': 'FHA Loan Units', 'FHA_2_AssistedUnits':
                                 'FHA Loan Sub_Program Units', 'HOME_1_AssistedUnits':'HOME Grants','Census Tract':'GEOID'
                                },inplace=True)

# atlanta_subset = subset_income_pd.loc[subset_income_pd["City"]== "Atlanta"]

#Keep only atlanta GEOCODES
atlanta_subset_nhpd = atlantageocodes.merge(subset_income_pd, how='inner', on='GEOID')
display(atlanta_subset_nhpd.head())

,GEOID,NHPD Property ID,PropertyStatus,Property Name,Property Address,City,State,Zip,Active Subsidies,TargetTenantType,...,EarliestStartDate,Total Units,Section 8 Housing Units,Supportive Housing Elderly,FHA Loan Units,FHA Loan Sub_Program Units,Low Income Housing Tax Credit,Low Income Housing Tax Credit sub-program,HOME Grants,Rural Rental Housing
0,13013180103,1105346,Active,BRASELTON COURT,1018 Beaver Dam Rd,Hoschton,GA,30548,1,NaN,...,1/1/2014,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13013180103,1127409,Active,NaN,1911 Highway 211 NW,Hoschton,GA,30548-3526,1,NaN,...,11/9/2015,67,NaN,NaN,NaN,NaN,NaN,NaN,67.0,NaN
2,13013180104,1102155,Active,AUTRY PINES SR VILLAGE,155 Autry Rd,Auburn,GA,30011-3206,2,NaN,...,1/1/2015,64,NaN,NaN,NaN,NaN,0.0,64.0,64.0,NaN
3,13013180204,1000412,Active,INNOVATION PROPERTIES,56 MAYNARD ST,WINDER,GA,30680,1,NaN,...,2/1/2016,40,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13013180204,1037926,Active,WINDING HOLLOW,174 S BROAD ST,WINDER,GA,30680,1,Elderly,...,12/1/2018,40,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN



# The following code imports all the dataset into a SQLite

In [154]:
#Create SQLite Connection
db_file = 'aaha.db'
conn = sqlite3.connect(db_file)

#Add DataFrames to DB
geotableclean.to_sql('censustract', con=conn, if_exists='replace', index=False)
summary.to_sql('census_dataset', con=conn, if_exists='replace', index=False)
clean_HUDcsv.to_sql('hud_dataset', con=conn, if_exists='replace', index=False)
atlanta_subset_nhpd.to_sql('nhpd_dataset', con=conn, if_exists='replace', index=False)
